In [10]:
import itertools
import re
import json
import random
import time

In [4]:
class CNF:
    def __init__(self, dimacs):
        dimacs_tokens = re.split('\s+', dimacs)
        self.num_of_vars = int(dimacs_tokens[2])
        self.num_of_clauses = int(dimacs_tokens[3])
        clauses_dimacs = [int(x) for x in dimacs_tokens[4:]]
        self.clauses = [list(clause) for is_zero, clause 
                    in itertools.groupby(clauses_dimacs, lambda x: x == 0) 
                    if not is_zero]
        
    def evaluate(self, valuation):
        for clause in self.clauses:
            clause_sat = False
            for literal in clause:
                val_idx = abs(literal) - 1
                if literal > 0 and valuation[val_idx] or literal < 0 and not valuation[val_idx]:
                    clause_sat = True
                    break
            if not clause_sat:
                return False
        return True

In [5]:
class Problem:
    def __init__(self, file_name):
        try:
            with open(file_name, 'r') as f:
                dimacs_list = json.load(f)
                self.cnf_list = [CNF(dimacs) for dimacs in dimacs_list]
                self.num_of_vars = self.cnf_list[0].num_of_vars
        except IOError:
            print(f'Error opening file {file_name}')
            exit(1)

In [29]:
class Solution:
    def __init__(self, problem, code=None):
        self.problem = problem
        if code is None:
            self.code = random.choices([True, False], k=self.problem.num_of_vars)
        else:
            self.code = code
        self.fitness = self.calc_fitness()
        
    def __lt__(self, other):
        return self.fitness < other.fitness
    
    def calc_fitness(self):
        num_of_sat = 0
        for formula in self.problem.cnf_list:
            if formula.evaluate(self.code):
                num_of_sat += 1
        return 1 / (num_of_sat + 1)
    
    def get_num_sat(self):
        return round(1 / self.fitness - 1)

In [65]:
class BFSolver:
    @staticmethod
    def solve(problem):
        solutions = [Solution(problem, valuation) for valuation in 
                     itertools.product([True, False], repeat=problem.num_of_vars)]
        return max(solutions)

In [8]:
class GeneticSolver:
    def __init__(self, num_of_generations=10, population_size=10, mutation_probability=0.2):
        self.num_of_generations = num_of_generations
        self.population_size = population_size
        self.mutation_probability = mutation_probability

    def _generate_population(self, problem):
        chromosome_dim = problem.num_of_vars
        return [[random.choice([True, False]) for _ in range(chromosome_dim)] for _ in range(self.population_size)]

    def _fitness(self, problem, chromosome):
        num_of_sat = 0
        for formula in problem.cnf_list:
            if formula.evaluate(chromosome):
                num_of_sat = num_of_sat + 1
        return num_of_sat

    def _selection(self, population, fitness_list):
        # FIX maybe better _selection, or lower mutation
        return random.choices(population, weights=[1 / (fitness + 1) for fitness in fitness_list], k=1)[0]

    def _crossover(self, parent_1, parent_2):
        bp = random.randrange(len(parent_1))
        child_1 = parent_1[:bp] + parent_2[bp:]
        child_2 = parent_2[:bp] + parent_1[bp:]
        return child_1, child_2

    def _mutate(self, chromosome):
        if random.random() < self.mutation_probability:
            index = random.randrange(len(chromosome))
            chromosome[index] = not chromosome[index]

    def solve(self, problem):
        population = self._generate_population(problem)
        fitness_list = [self._fitness(problem, chromosome) for chromosome in population]
        min_sat = min(fitness_list)
        new_population = [None for _ in range(len(population))]
        for i in range(self.num_of_generations):
            for j in range(self.population_size // 2):
                parent_1 = self._selection(population, fitness_list)
                parent_2 = self._selection(population, fitness_list)
                child_1, child_2 = self._crossover(parent_1, parent_2)
                self._mutate(child_1)
                self._mutate(child_2)
                new_population[2 * j] = child_1
                new_population[2 * j + 1] = child_2
            population = new_population
            fitness_list = [self._fitness(problem, chromosome) for chromosome in population]
            min_sat = min(fitness_list + [min_sat])
            print(f'generation[{i}] :  {min_sat}')
        return min_sat

In [63]:
p = Problem('problem_instances/1.json')
bf_solver = BFSolver()
start_time = time.time()
result = BFSolver.solve(p)
print(f'{result.code} {result.get_num_sat()}')
print(time.time() - start_time)
#ga_solver = GeneticSolver()
#print(ga_solver.solve(p))

(True, True, True, False, False, False, False, False, True, True) 30
0.12578558921813965
